# Sistemas de ecuaciones lineales

En el notebook anterior, vimos que un problema de valores en la frontera, por ejemplo, el problema de Poisson, se puede expresar como un sistema de ecuaciones lineales, y, por lo tanto, se puede escribir en forma matricial como

$$ \mathsf{A} \cdot \mathbf{V} = \mathbf{b},$$

donde $\mathbf{V} = (V_0, \ldots, V_L)$ son los valores desconocidos del potencial, $\mathsf{A}$ es una matriz proveniente de la discretización del Laplaciano usando diferencias finitas, y $\mathbf{b}$ es un vector que combina el potencial y las condiciones en la frontera.

En el notebook anterior, utilizamos un método **iterativo** para resolver este sistema, pero vimos que convergía lentamente a la solución. 

En este notebook, veremos un método **directo** (es decir, no iterativo, y que requiere un número de pasos finito para terminar el cálculo) para resolver sistemas de ecuaciones lineales, el método de **eliminación gaussiana**, que seguramente ya vieron en Álgebra Lineal.

## Eliminación gaussiana

Considera el sistema de ecuaciones lineales

$$ \mathsf{A} \cdot \mathbf{x} = \mathbf{b}. \qquad (1)$$

Aquí, $\mathsf{A}$ es una matriz cuadrada dada, de tamaño $n \times n$, y $\mathbf{b}$ es un vector dado en $\mathbb{R}^n$. Queremos resolver esta ecuación para encontrar el vector desconocido $\mathbf{x} \in \mathbb{R}^n$.

Recuerda que podemos resolver este sistema usando el método de eliminación gaussiana, que consiste en llevar a cabo operaciones de renglón, tanto para la matriz $\mathsf{A}$ como el vector $\mathbf{b}$, para reducir $\mathsf{A}$ a una forma triangular.

[En este notebook, debes implementar los métodos numéricos "a mano" en la computadora, sin utilizar ningún método ni paquete de Julia ya implementado.]

In [51]:
using Interact, Plots
gr()

Plots.GRBackend()

**[1]** Re-escribe la ecuación (1) utilizando índices y sumas.

**[2]** Considera la ecuación $\mathsf{U} \cdot \mathbf{x} = \mathbf{c}$, en la cual $\mathsf{U}$ es una matriz triangular superior ("*U*pper triangular").

(i) ¿Cuál es la condición para que una matriz $\mathsf{A}$ sea triangular superior, en términos de sus entradas $A_{ij}$? 

(ii) ¿Cómo se puede resolver esta ecuación? Piensa primero en un caso chiquito, e.g. $3 \times 3$ con entradas $U_{11}$ etc., para que puedas hacer todo explícitamente.

(iii) Implementa este método para una matriz triangular superior arbitraria.

(iv) Verifica que tu método numérico funciona, es decir ¡que el resultado sí sea una solución de la ecuación original!

### [1]

La el problema matricial se puede escribir como

$$\sum_{j=1}^{N} A_{ij}x_j=b_i$$

En notación de índices (utilizando la convención de Einstein), escribimos exactamente lo mismo y quitamos el símbolo de suma.

$$A_{ij}x_j=b_i$$


### [2]

Una matriz $\mathbf{A}$ es triangular superior si y solo si $\forall i>j, A_{ij}=0$

In [52]:
function evaluar(A::Array{Float64,2},B::Array{Float64,1};test=false)
    if size(A)[1]!=size(A)[2] 
        error("La matriz no es cuadrada")
    end
    N=size(A)[1]
    total=zeros(N)
    total[N]=B[N]/A[N,N]
    for i in 1:(N-1)
        i=(N-i)
        sum=0.0
        for k in i:N
            if test==true
                @show k
                @show A[i,k],total[k] 
                @show sum
            end
            sum=sum+A[i,k]*total[k]
        end
        if test==true
            @show i
            @show total
            @show (B[i],sum,A[i,i])
        end
        total[i]=(B[i]-sum)/A[i,i]
    end
    return total
end

evaluar (generic function with 1 method)

In [53]:
M=[1.0 1.0 1.0; 0.0 2.0 1.0; 0.0 0.0 1.0]
B=[3.0,2.0,1.0]
evaluar(M,B)

Array{Float64, 2}, Array{Float64, 1}

3-element Array{Float64,1}:
 1.5
 0.5
 1.0

**[3]** (i) Escribe, a mano, el método de eliminación gaussiana para reducir una matriz de $2 \times 2$ a una matriz triangular superior, en términos de operaciones de renglón. ¿Qué se debe hacer al lado derecho de la ecuación? Supón (por el momento) que no hay problemas con dividir por cero. 

(ii) Impleméntalo numéricamente. Utiliza tu código para resolver el sistema de ecuaciones lineales

\begin{align} 
    3x + 7y &= 1; \\
    2x + 4y &= 1.
\end{align}

Por supuesto, ¡debes verificar que la solución que encuentres realmente sí sea solución de la ecuación!

In [54]:
function gauss2x2(M::Array{Float64,2},B::Array{Float64,1};test=false)
    if M[2,1]==0.0
        error("la entrada 2,1 no puede ser cero")
    end
    M=hcat(M,B)
    M[2,:]=M[2,:]-(M[2,1]/M[1,1])*M[1,:]
    C=M[1:2,1:2]
    D=M[1:2,3]
    return evaluar(C,D)
end  

) in module Main at In[19]:2 overwritten at In[52]:2.

gauss2x2 (generic function with 1 method)

In [55]:
M=[3.0 7.0; 2.0 4.0]
B=[1.0,1.0]
gauss2x2(M,B)

, Array{Float64, 2}, Array

2-element Array{Float64,1}:
  1.5
 -0.5

**[4]** (i) Implementa el método de eliminación gaussiana para una matriz de tamaño $3 \times 3$,  y luego para una matriz arbitraria, para resolver la ecuación (1). Para eso, escribe una función que reduzca una matriz arbitraria a una triangular superior, y luego utiliza la función que ya tienes para resolver el sistema triangular superior. [Sigue suponiendo que no haya divisiones entre 0.]

(ii) Utiliza tu código para resolver el sistema

\begin{align}
    2x + y -  z &=   8; \\
   -3x - y + 2z &= -11; \\
   -2x + y + 2x &=  -3.
\end{align}

In [56]:
function gauss(M::Array{Float64,2},B=Array{Float64,1};test=false)
    if size(M)[1]!=size(M)[2]
        error("la matriz debe ser cuadrada")
    end
    N=size(M)[1]
    M=hcat(M,B)
    for j in 1:(N-1)
        if test==true 
            @show j
        end
        for l in 1:(N-j)
            i=N+1-l
            if test==true
                @show i
                @show M[i,j]
                @show M[i-1,j]
                @show M
            end
            if M[i-1,j]==0
                M[i-1,:],M[i,:]=M[i,:],M[i-1,:]
                continue
            end
            M[i,:]=M[i,:]-(M[i,j]/M[i-1,j])*M[i-1,:]
        end
    end
    A=M[:,1:N]
    B=M[:,end]
    X=evaluar(A,B)
    return X
end

{Float64, 1}) in module Main overwritten.


gauss (generic function with 2 methods)

In [57]:
M=[3.0 7.0; 2.0 4.0]
B=[1.0,1.0]
gauss(M,B)

2-element Array{Float64,1}:
  1.5
 -0.5

In [58]:
M=[2.0 1.0 -1.0; -3.0 -1.0 2.0; -2.0 1.0 2.0]
B=[8.0,-11.0,-3.0]
X=gauss(M,B,test=true)

j = 1
i = 3
M[i,j] = -2.0
M[i - 1,j] = -3.0
M = [2.0 1.0 -1.0 8.0; -3.0 -1.0 2.0 -11.0; -2.0 1.0 2.0 -3.0]
i = 2
M[i,j] = -3.0
M[i - 1,j] = 2.0
M = [2.0 1.0 -1.0 8.0; -3.0 -1.0 2.0 -11.0; 0.0 1.66667 0.666667 4.33333]
j = 2
i = 3
M[i,j] = 1.6666666666666665
M[i - 1,j] = 0.5
M = [2.0 1.0 -1.0 8.0; 0.0 0.5 0.5 1.0; 0.0 1.66667 0.666667 4.33333]


Array{Float64, 1}) in module Main at In[21]:2 overwritten at In[54]:2.

3-element Array{Float64,1}:
  2.0
  3.0
 -1.0

In [59]:
M=[1.0 1.0 1.0; 2.0 0.0 1.0; 0.0 1.0 0.0]
B=[1.0,1.0,1.0]
X=gauss(M,B)

 overwritten.

3-element Array{Float64,1}:
  1.0
  1.0
 -1.0

### [Extra]

Podemos notar que para encontrar raíces de una función determinada, simplemente necesitamos aproximar la función con una serie de taylor vectorial.

$$f(\vec{x}+\vec{\delta}) = f(\vec{x}) + J_f[\vec{x}]\vec{\delta}=0 \implies \vec{\delta}= - J_f^{-1}f(\vec{x})$$

Sin embargo, sabemos que para un sistema de ecuaciones lineales de la forma:

$$J_f \: \vec{x}=f(\vec{x_n})$$

La solución será:

$$\vec{x} = J_f^{-1} \: f(\vec{x_n})$$

Así, para computar esa expresión en el método de Newton, podemos simplemente resolver ese sistema de ecuaciones en cada paso.

In [60]:
function devpar(f::Function,x0::Array{Float64,1}, i::Int64,j::Int64; h::Float64=1e-8)
    dim=length(x0)
    H=zeros(dim)
    H[j]=h
    dev=(f(x0+H)-f(x0-H))/(2*h)
    return dev[i]
end
function jacobiano(f::Function,x0::Array{Float64,1};h=1e-8)
    dim=length(x0)
    return [devpar(f,x0,i,j,h=h) for i in 1:dim, j in 1:dim]
end
function newton(f::Function,x0::Array{Float64};tol=1e-15,test=false)
    i=1
    X=copy(x0)
    while norm(f(X),Inf)>tol
        M=jacobiano(f,X)
        B=f(X)
        A=gauss(M,B)
        X=X-A
        if test==true
            @show i
            @show M
            @show B
            @show A
            @show X
        end
        i=i+1
    end
    return X
end

, 2}, Any) in module Main overwritten.

newton (generic function with 3 methods)

In [61]:
h(A)=[A[1]^2 - 2 , A[2]^2 - 3]

Function, Array{Float64, 1}, Int64, Int64)

h (generic function with 1 method)

In [62]:
newton(h,[1.5,1.5],test=false)

 in module Main at In[48]:2 overwritten at In[60]:2.

2-element Array{Float64,1}:
 1.41421
 1.73205

Main.

**[5]** (i) El acto de llevar a cabo una operación de renglón (como las que se utilizan en la eliminación gaussiana) sobre una matriz $\mathsf{A}$ se puede escribir como una multiplicación de $\mathsf{A}$ con otra matriz $\mathsf{L}$. ¿Cuál matriz $\mathsf{L}$? ¿Qué propiedad tiene $\mathsf{L}$?


(ii) Considerando que el método de eliminación gaussiana consiste en una secuencia de operaciones de renglón, que son multiplicaciones por matrices $L_j$, y que produce una matriz triangular superior $U$, ¿cómo se puede escribir $U$ en términos de las $L_j$?

(iii) Por lo tanto, ¿de qué forma se puede escribir $\mathsf{A}$? 

(iv) ¿Qué implica esto si un problema consiste en resolver muchas ecuaciones de la forma $\mathsf{A} \cdot \mathbf{x}^{(i)} = \mathbf{b}^{(i)}$, para distintos lados derechos $\mathbf{b}^{(i)}$?